# 概述

本notebook主要介绍如何进行中文OCR算法建模，它主要包含三个环节，分别是文字角度识别、文字检测、文字识别三个部分。致谢 https://github.com/xiaofengShi/CHINESE-OCR, 本文源码可以从https://github.com/backyes/CHINESE-OCR 获取

# 准备环境

## 新建notebook，使用 hub.cn-east-p1.netease.com/staffai/notebook:tf-1.14.0-gpu-py3-ocr 创建Jupyter Notebook服务，参见交互式建模手册

> 该镜像基于TensorFlow环境并安装了若干算法依赖， 无需额外操作

    
```bash
apt-get install -y python-pydot python-pydot-ng graphviz cuda-cusparse-dev-10-0 cuda-cublas-dev-10-0  cmake vim cuda-curand-dev-10-0
pip install pydot==1.4.1 graphviz==0.14.2
pip install easydict -i https://pypi.tuna.tsinghua.edu.cn/simple/ ##选择国内源，速度更快
pip install keras==2.0.8  -i https://pypi.tuna.tsinghua.edu.cn/simple/  
pip install Cython opencv-python -i https://pypi.tuna.tsinghua.edu.cn/simple/ 
pip install matplotlib -i https://pypi.tuna.tsinghua.edu.cn/simple/ 
pip install -U pillow -i https://pypi.tuna.tsinghua.edu.cn/simple/
pip install  h5py lmdb mahotas -i https://pypi.tuna.tsinghua.edu.cn/simple/
pip install futures==3.1.1 -i https://pypi.tuna.tsinghua.edu.cn/simple/
pip install torch==1.0.1  torchvision==0.2.0 
pip install futures==3.1.1 -i https://pypi.tuna.tsinghua.edu.cn/simple/
pip install tensorflow==1.13.1 tensorflow-gpu==1.13.1
pip install warpctc-pytorch10-cuda90==0.1.3
```


## 下载 https://github.com/backyes/CHINESE-OCR 

In [1]:
!git clone https://github.com/backyes/CHINESE-OCR

Cloning into 'CHINESE-OCR'...
remote: Enumerating objects: 42, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (34/34), done.
^Cceiving objects:   9% (49/514), 972.01 KiB | 52.00 KiB/s   


## 准备基本环节环境

In [2]:
!cd ./ctpn/lib/utils && sh make.sh

/usr/local/lib/python3.6/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /home/jovyan/CHINESE-OCR/ctpn/lib/utils/bbox.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
/usr/local/lib/python3.6/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /home/jovyan/CHINESE-OCR/ctpn/lib/utils/cython_nms.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)
/usr/local/lib/python3.6/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /home/jovyan/CHINESE-OCR/ctpn/lib/utils/gpu_nms.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)

Error compiling Cython file:
------------------------------------------------------------
...
# ---------------

# 准备数据

从 链接: https://pan.baidu.com/s/1vWa604CEGJZNEpys0K9N7A 提取码: p3xy， 将下载的数据解压到代码工程根目录， 解压后目录如下：

```bash
/home/jovyan/HINESE-OCR
total 540492
-rw-r--r-- 1 jovyan users       3520 Dec 14 21:00 CHINESE_OCR.ipynb
-rw-r--r-- 1 jovyan users       1052 Dec 14 20:58 Dockerfile
-rw-r--r-- 1 jovyan users      11346 Dec  8 11:13 README.md
-rw-r--r-- 1 jovyan users  553431812 Dec  9 15:39 VGG_imagenet.npy
drwxr-xr-x 2 jovyan users       3611 Dec  9 18:37 __pycache__
drwxr-xr-x 3 jovyan users       1878 Dec 14 10:24 angle
drwxr-xr-x 2 jovyan users     121748 Dec  8 11:13 asset
-rw-r--r-- 1 jovyan users        404 Dec 14 10:24 conf.py
drwxr-xr-x 4 jovyan users      30309 Dec 14 10:24 crnn
drwxr-xr-x 9 jovyan users  241743417 Dec 14 10:24 ctpn
drwxr-xr-x 4 jovyan users 2940690554 Dec  9 20:48 datasets
drwxr-xr-x 3 jovyan users      20490 Dec 14 10:24 ocr
-rw-r--r-- 1 jovyan users       5160 Dec  9 13:12 pascal_voc.py
drwxr-xr-x 6 jovyan users   56552660 Dec 14 10:24 predict
-rwxr-xr-x 1 jovyan users        433 Dec 14 10:24 scripts.sh
-rwxr-xr-x 1 jovyan users       3249 Dec 14 10:24 setup.sh
drwxr-xr-x 8 jovyan users   11725974 Dec 14 10:24 train
```

其中

```
drwxr-xr-x 4 jovyan users 2940690554 Dec  9 20:48 datasets
```
datasets是准备的数据集以及一些预训练模型，下面分别介绍

- VOCdevkit  用于训练检测模型的数据集

- checkpoints detection模型预训练的模型checkpoints

- modelAngle.h5 用于对文字方向进行推断的预训练模型

- my_model_keras.h5 用于识别模型的预训练模型（Keras 模型）

- netCRNN_pytorch.pth 用于识别的预训练模型（PyTorch模型），这个模型本demo里没有使用，

下面查看配置文件，主要配置了数据集以及预训练模型的路径，如下:

```bash
tf-docker ~/CHINESE-OCR > cat  conf.py
angel_model = '/home/jovyan/CHINESE-OCR/datasets/modelAngle.h5'
# ctpn
devkit_path = '/home/jovyan/CHINESE-OCR/datasets/VOCdevkit/'
ctpn_pretrained_model = '/home/jovyan/CHINESE-OCR/datasets/VGG_imagenet.npy'
ctpn_ckpt_path = '/home/jovyan/CHINESE-OCR/ctpn/output/ctpn_end2end/voc_2007_trainval'
ctpn_ckpt_model = '/home/jovyan/CHINESE-OCR/ctpn/output/ctpn_end2end/voc_2007_trainval/VGGnet_fast_rcnn_iter_180000.ckpt'
#crnn
ocr_model = '/home/jovyan/CHINESE-OCR/datasets/my_model_keras.h5'
netCRNN_pytorch_pth = '/home/jovyan/CHINESE-OCR/datasets/netCRNN_pytorch.pth'
```

请跟进实际datasets绝对路径修改此上面变量。

In [3]:
!cat conf.py

angel_model = '/home/jovyan/CHINESE-OCR/datasets/modelAngle.h5'
# ctpn
devkit_path = '/home/jovyan/CHINESE-OCR/datasets/VOCdevkit/'
ctpn_pretrained_model = '/home/jovyan/CHINESE-OCR/datasets/VGG_imagenet.npy'
ctpn_ckpt_path = '/home/jovyan/CHINESE-OCR/ctpn/output/ctpn_end2end/voc_2007_trainval'
ctpn_ckpt_model = '/home/jovyan/CHINESE-OCR/ctpn/output/ctpn_end2end/voc_2007_trainval/VGGnet_fast_rcnn_iter_180000.ckpt'
#crnn
ocr_model = '/home/jovyan/CHINESE-OCR/datasets/my_model_keras.h5'
netCRNN_pytorch_pth = '/home/jovyan/CHINESE-OCR/datasets/netCRNN_pytorch.pth'



#  训练模型

##  训练检测模型

下面主要介绍重新fine tuning 检测模型以及识别模型。 注意你需要一个内存足够匹配数据集的Jupyter Notebook否则可能无法加载数据。如果要使用算法平台分布式训练能力请参考其他文档。

In [5]:
# transfer learning detection model
!PYTHONPATH=./ python ctpn/ctpn/train_net.py # 重新基于/home/jovyan/CHINESE-OCR/datasets/VOCdevkit/ 数据集微调模型

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

## 测试检测模型

In [15]:
# 用demo测试这个检测模型
!PYTHONPATH=./ python ctpn/ctpn/demo.py

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

展示测试结果


![test](ctpn/data/results/009.jpg)

## 训练识别模型

1. 数据集准备过程可以参见 train/keras_train/create_dataset，这里不赘述。

2. 主要利用keras + TensorFlow 构建的crnn识别模型，基于/home/jovyan/CHINESE-OCR/datasets/my_model_keras.h5 模型进行transfer learning训练， 训练输出的模型在 `/home/jovyan/CHINESE-OCR/train/keras_train/output`, 详细训练参数可以参见 `train/keras_train/trainbatch.py` 。

In [16]:
!PYTHONPATH=./ python train/keras_train/trainbatch.py

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

#  端到端的测试模型

主要利用训练出的模型进行最终的测试评估工作。

测试数据位于 `/home/jovyan/CHINESE-OCR/predict/test`, `predict/demo.py` 文件会对测试数据进行逐个测试，并输出识别结果。 相对模型微调请实际分析相关底层代码，此处不赘述细节，仅做demo。

```python
tf-docker ~/CHINESE-OCR/predict > cat demo.py
# coding:utf-8

import os
import time
from glob import glob

import numpy as np
from PIL import Image

from predict import model

paths = glob(os.path.dirname(__file__) + '/test/*.*')

if __name__ == '__main__':
    im = Image.open(os.path.dirname(__file__) + "/test/test_pre.png")
    img = np.array(im.convert('RGB'))
    t = time.time()
    '''
    result,img,angel分别对应-识别结果，图像的数组，文字旋转角度
    '''
    result, img, angle = model.model(
        img, model='keras', adjust=True, detectAngle=True)
    print("It takes time:{}s".format(time.time() - t))
    print("---------------------------------------")
    for key in result:
        print(result[key][1])

    im = Image.open(os.path.dirname(__file__) + "/test/img/result.png")
    img = np.array(im.convert('RGB'))
    t = time.time()
    '''
    result,img,angel分别对应-识别结果，图像的数组，文字旋转角度
    '''
    result, img, angle = model.model(
        img, model='keras', adjust=True, detectAngle=True)
    print("It takes time:{}s".format(time.time() - t))
    print("---------------------------------------")
    for key in result:
        print(result[key][1])
```

In [19]:
!ls predict/test/*

predict/test/001.jpg				 predict/test/IMG_0689.JPG
predict/test/002.jpg				 predict/test/IMG_1563.JPG
predict/test/003.jpg				 predict/test/WechatIMG305.jpeg
predict/test/004.jpg				 predict/test/ctpn____.png
predict/test/005.jpg				 predict/test/demo-card-1.jpeg
predict/test/006.jpg				 predict/test/qqqqqq.png
predict/test/007.jpg				 predict/test/test.jpeg
predict/test/008.jpg				 predict/test/test.png
predict/test/009.jpg				 predict/test/test1.png
predict/test/010.png				 predict/test/test2.png
predict/test/1.png				 predict/test/test_ctpn.png
predict/test/2.png				 predict/test/test_pre.png
predict/test/3.png				 predict/test/test_result.png
predict/test/4.jpg				 predict/test/ttttt.png
predict/test/CgQgIll5TaGADkiiAAFhxGXefcA951.jpg  predict/test/ttttt_result.png
predict/test/CgQgIlljJlyAF3fGAABk3TG5RM4164.jpg

predict/test/img:
result.png  tmp.jpg  tmp.png  tmp1.jpg	tmp1.png


测试2个图片： 

- predict/test/test_pre.png

![test](predict//test/test_pre.png)

- predict/test/img/result.png

![test](predict/test/img/result.png)

In [20]:
!PYTHONPATH=./ python predict/demo.py

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 